In [6]:
print("Start")

Start


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.preprocessFile import preprocessFile

In [8]:
from scipy.io import loadmat
import os

In [9]:
sig_file=[]
data_file=[]

In [10]:
for file in sorted(os.listdir("dataset")):
    file_path=os.path.join("dataset",file)
    if file.endswith(".hea"):
        data_file.append(file_path)
    else:
        sig_file.append(file_path)


In [11]:
import numpy as np
from scipy.signal import stft, windows

In [13]:
all_ehgdata=[]
labels=[]
for file_hea,file_mat in zip(data_file,sig_file):
    meta_data,label,downsample_signal, t_downsample=preprocessFile(file_hea,file_mat)
    if meta_data is not None:
        labels.append(label[0])
        # 1. Cắt tín hiệu về đúng 30 phút
        max_time = 30 * 60  # 30 phút = 1800 giây
        if t_downsample[-1] > max_time:
            idx = np.argmin(np.abs(t_downsample - max_time))
            t_downsample = t_downsample[:idx+1]
            # MATLAB chọn kênh số 3 => Python là chỉ số 2
            downsample_signal = downsample_signal[:idx+1, 2]
        else:
            downsample_signal = downsample_signal[:, 2]

        #2. Tính tần số lấy mẫu
        fs = 1.0 / (t_downsample[1] - t_downsample[0])

        #3. Tính chiều dài cửa sổ
        f0 = 0.1
        window_length = round(fs / f0 * 6)
        window = windows.hamming(window_length)

        #4. Tính STFT
        f, t_spec, Zxx = stft(
            downsample_signal,
            fs=fs,
            window=window,
            nperseg=window_length,
            noverlap=window_length//2,
            boundary=None,
            padded=False
        )

        # 5. Biến về log(abs)
        s = np.log(np.abs(Zxx) + 1e-8)   # tránh log(0)

        # 6. Zero-pad theo thời gian (cột)
        target_cols = 106
        if s.shape[1] < target_cols:
            pad_cols = target_cols - s.shape[1]
            s = np.hstack([s, np.zeros((s.shape[0], pad_cols))])

        #7. Lấy nửa phổ dương
        mid = s.shape[0] // 2
        s_half = s[mid:, :target_cols]
        all_ehgdata.append(s_half)  
     

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

# all_ehgdata.shape = (159, 151, 106)
# labels.shape = (159,)
# Convert sang tensor
X = torch.tensor(all_ehgdata, dtype=torch.float32) 
y = torch.tensor(labels, dtype=torch.long)         